In [1]:
import pandas as pd
df = pd.read_csv(r"runs\robustness_quickdemo\summary.csv")
df.sort_values("sharpe", ascending=False).head(10)


,run,loo,lookbacks,w_tsmom,w_xsec,w_mr,w_volcarry,vc_top_q,vc_bot_q,vc_lookback,target_vol,vol_lookback,max_leverage,cagr,vol,sharpe,maxdd,mar
4,5,-,"(63,126,252)",1.0,0.8,0.6,0.0,0.25,0.25,42,0.15,15,2.0,0.083124,0.035289,2.355510,-0.025114,3.309813
12,13,-,"(63,126,252)",1.0,0.8,0.6,0.0,0.25,0.25,84,0.15,15,2.0,0.083124,0.035289,2.355510,-0.025114,3.309813
6,7,-,"(63,126,252)",1.0,0.8,0.6,0.0,0.25,0.25,42,0.15,30,2.0,0.068909,0.036144,1.906529,-0.024975,2.759124
14,15,-,"(63,126,252)",1.0,0.8,0.6,0.0,0.25,0.25,84,0.15,30,2.0,0.068909,0.036144,1.906529,-0.024975,2.759124
5,6,-,"(63,126,252)",1.0,0.8,0.6,0.0,0.25,0.25,42,0.15,15,3.0,0.073906,0.039015,1.894317,-0.032041,2.306626
13,14,-,"(63,126,252)",1.0,0.8,0.6,0.0,0.25,0.25,84,0.15,15,3.0,0.073906,0.039015,1.894317,-0.032041,2.306626
0,1,-,"(63,126,252)",1.0,0.8,0.6,0.0,0.25,0.25,42,0.10,15,2.0,0.045736,0.025695,1.779986,-0.021725,2.105193
8,9,-,"(63,126,252)",1.0,0.8,0.6,0.0,0.25,0.25,84,0.10,15,2.0,0.045736,0.025695,1.779986,-0.021725,2.105193
7,8,-,"(63,126,252)",1.0,0.8,0.6,0.0,0.25,0.25,42,0.15,30,3.0,0.057508,0.039819,1.444226,-0.032432,1.773190
15,16,-,"(63,126,252)",1.0,0.8,0.6,0.0,0.25,0.25,84,0.15,30,3.0,0.057508,0.039819,1.444226,-0.032432,1.773190


In [3]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from pathlib import Path

root = Path(r"C:\Users\speed\Desktop\Forex CFD's system\data")
is_eq = pd.read_csv(root/"IS_candidate_equity.csv", parse_dates=['ts']).set_index('ts')['portfolio_equity']
oos_eq = pd.read_csv(root/"OOS_candidate_equity.csv", parse_dates=['ts']).set_index('ts')['portfolio_equity']

def metrics(port):
    r = port.pct_change().dropna()
    years = (port.index[-1]-port.index[0]).days/365.25
    cagr = port.iloc[-1]**(1/years)-1 if years>0 else np.nan
    vol = r.std()*np.sqrt(252) if len(r)>2 else np.nan
    dd = (port/port.cummax()-1).min() if len(port)>1 else np.nan
    sharpe = cagr/vol if isinstance(vol,float) and vol>0 else np.nan
    mar = cagr/abs(dd) if isinstance(dd,float) and dd<0 else np.nan
    return dict(CAGR=cagr, VOL=vol, Sharpe=sharpe, MaxDD=dd, MAR=mar)

print("IS:", metrics(is_eq))
print("OOS:", metrics(oos_eq))

fig = go.Figure()
fig.add_trace(go.Scatter(x=is_eq.index, y=is_eq, name="IS"))
fig.add_trace(go.Scatter(x=oos_eq.index, y=oos_eq, name="OOS"))
fig.update_layout(title="Equity: IS vs OOS", xaxis_title="Date (UTC)", yaxis_title="Equity (base=1.0)")
fig.show()


IS: {'CAGR': np.float64(0.03922601120416469), 'VOL': np.float64(0.0374518535810454), 'Sharpe': np.float64(1.047371690676939), 'MaxDD': np.float64(-0.032431778581143766), 'MAR': np.float64(1.2094930626768392)}
OOS: {'CAGR': np.float64(0.03640542850518913), 'VOL': np.float64(0.0891755704040629), 'Sharpe': np.float64(0.40824441425193814), 'MaxDD': np.float64(-0.07390842753675086), 'MAR': np.float64(0.49257479449263863)}
